## the start of encoding is sometimes ENCODING_START, ORIENT_OFF, ORIENT_END, or maybe something else
## So instead go from COUNTDOWN_END and find the index that consistently gives you the first WORD to get the right evs for encoding start time

In [1]:
import pandas as pd; pd.set_option('display.max_columns', 30)
import numpy as np
from cmlreaders import CMLReader, get_data_index
from ptsa.data.filters import ButterworthFilter
import sys
import os
import matplotlib.pyplot as plt
%matplotlib inline
from pylab import *
from copy import copy
from scipy import stats
import pickle
plt.rcParams['pdf.fonttype'] = 42; plt.rcParams['ps.fonttype'] = 42 # fix fonts for Illustrator
sys.path.append('/home1/john/Downloads/ptsa_plot-master/')
sys.path.append('/home1/john/johnModules')
from brain_labels import MTL_labels, LTC_labels, PFC_labels, OTHER_labels, ALL_labels # all location labels
%load_ext autoreload
%autoreload
from general import *
from SWRmodule import *

df = get_data_index("r1") # all RAM subjects
exp = 'FR1' # 'FR1' 'catFR1'

/home1/john/anaconda3/envs/env1/lib/python3.7/site-packages/ptsa/data/timeseries.py:526: FutureWarning: xarray subclass TimeSeriesX should explicitly define __slots__
  class TimeSeriesX(TimeSeries):


In [2]:
# 575 FR sessions. first 18 of don't load so skip those 
exp_df = df[df.experiment==exp]
if exp == 'FR1':
    exp_df = exp_df[
                    ((df.subject!='R1015J') | (df.session!=0)) & 
                    ((df.subject!='R1063C') | (df.session!=1)) & 
                    ((df.subject!='R1093J') | (~df.session.isin([1,2]))) &
                    ((df.subject!='R1100D') | (~df.session.isin([0,1,2]))) &
                    ((df.subject!='R1120E') | (df.session!=0)) &
                    ((df.subject!='R1122E') | (df.session!=2)) &
                    ((df.subject!='R1154D') | (df.session!=0)) &
                    ((df.subject!='R1186P') | (df.session!=0)) &
                    ((df.subject!='R1201P') | (~df.session.isin([0,1]))) &
                    ((df.subject!='R1216E') | (~df.session.isin([0,1,2]))) &
                    ((df.subject!='R1277J') | (df.session!=0)) &
                    ((df.subject!='R1413D') | (df.session!=0)) & 
                    ((df.subject!='R1123C') | (df.session!=2)) & # artifacts that bleed through channels (see SWR FR1 prob sessions ppt)
                    ((df.subject!='R1151E') | (~df.session.isin([1,2]))) & # more bleed-through artifacts (see same ppt)
                    ((df.subject!='R1275D') | (df.session!=3))  # 3rd session an actual repeat of 2nd session (Paul should have removed from database by now)
    #                 (df.subject!='R1065J') # sub with 9000 trials
                   ] 
if exp == 'catFR1': 
    exp_df = exp_df[
                    ((df.subject!='R1044J') | (df.session!=0)) & # too few trials to do pg pairwise corr
                    ((df.subject!='R1491T') | (~df.session.isin([1,3,5]))) & # too few trials to do pg pairwise corr
                    ((df.subject!='R1486J') | (~df.session.isin([4,5,6,7]))) & # repeated data...will be removed at some point... @@
                    ((df.subject!='R1501J') | (~df.session.isin([0,1,2,3,4,5]))) & # these weren't catFR1 (and they don't load right anyway)
                    ((df.subject!='R1235E') | (df.session!=0)) & # split EEG filenames error...documented on Asana
                    ((df.subject!='R1310J') | (df.session!=1)) & # session 1 is just a repeat of session 0
                    ((df.subject!='R1239E') | (df.session!=0)) # some correlated noise (can see in catFR1 problem sessions ppt)
    ]
exp_df.shape
# exp_df = exp_df[257:] # for catFR1 this is R1385E-onwwards
# exp_df = exp_df[472:] # for FR1 this is R1385E-onwwards
# exp_df = exp_df[-19:]
exp_df

/home1/john/anaconda3/envs/env1/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


(565, 17)

Recognition                                         all_events  \
0            NaN  protocols/r1/subjects/R1001P/experiments/FR1/s...   
1            NaN  protocols/r1/subjects/R1001P/experiments/FR1/s...   
9            NaN  protocols/r1/subjects/R1002P/experiments/FR1/s...   
10           NaN  protocols/r1/subjects/R1002P/experiments/FR1/s...   
17           NaN  protocols/r1/subjects/R1003P/experiments/FR1/s...   
...          ...                                                ...   
2524         NaN  protocols/r1/subjects/R1565T/experiments/FR1/s...   
2534         NaN  protocols/r1/subjects/R1569T/experiments/FR1/s...   
2539         NaN  protocols/r1/subjects/R1571T/experiments/FR1/s...   
2543         NaN  protocols/r1/subjects/R1572T/experiments/FR1/s...   
2545         NaN  protocols/r1/subjects/R1573T/experiments/FR1/s...   

                                               contacts experiment  \
0     protocols/r1/subjects/R1001P/localizations/0/m...        FR1   
1     protocols/r1/subjects/R1001P/localizations/0/m...        FR1   
9     protocols/r1/subjects/R1002P/localizations/0/m...        FR1   
10    protocols/r1/subjects/R1002P/localizations/0/m...        FR1   
17    protocols/r1/subjects/R1003P/localizations/0/m...        FR1   
...                                                 ...        ...   
2524  protocols/r1/subjects/R1565T/localizations/0/m...        FR1   
2534  protocols/r1/subjects/R1569T/localizations/0/m...        FR1   
2539  protocols/r1/subjects/R1571T/localizations/0/m...        FR1   
2543  protocols/r1/subjects/R1572T/localizations/0/m...        FR1   
2545  protocols/r1/subjects/R1573T/localizations/0/m...        FR1   

     import_type  localization  \
0          build             0   
1          build             0   
9          build             0   
10         build             0   
17         build             0   
...          ...           ...   
2524       build             0   
2534       build             0   
2539       build             0   
2543       build             0   
2545       build             0   

                                            math_events  montage  \
0     protocols/r1/subjects/R1001P/experiments/FR1/s...        0   
1     protocols/r1/subjects/R1001P/experiments/FR1/s...        0   
9     protocols/r1/subjects/R1002P/experiments/FR1/s...        0   
10    protocols/r1/subjects/R1002P/experiments/FR1/s...        0   
17    protocols/r1/subjects/R1003P/experiments/FR1/s...        0   
...                                                 ...      ...   
2524  protocols/r1/subjects/R1565T/experiments/FR1/s...        0   
2534  protocols/r1/subjects/R1569T/experiments/FR1/s...        0   
2539  protocols/r1/subjects/R1571T/experiments/FR1/s...        0   
2543  protocols/r1/subjects/R1572T/experiments/FR1/s...        0   
2545  protocols/r1/subjects/R1573T/experiments/FR1/s...        0   

     original_experiment original_session  \
0                    NaN                0   
1                    NaN                1   
9                    NaN                0   
10                   NaN                1   
17                   NaN                0   
...                  ...              ...   
2524                 NaN              NaN   
2534                 NaN              NaN   
2539                 NaN              NaN   
2543                 NaN              NaN   
2545                 NaN              NaN   

                                                  pairs ps4_events  session  \
0     protocols/r1/subjects/R1001P/localizations/0/m...        NaN        0   
1     protocols/r1/subjects/R1001P/localizations/0/m...        NaN        1   
9     protocols/r1/subjects/R1002P/localizations/0/m...        NaN        0   
10    protocols/r1/subjects/R1002P/localizations/0/m...        NaN        1   
17    protocols/r1/subjects/R1003P/localizations/0/m...        NaN        0   
...                                                 ...        ...    

In [ ]:
# it's never both of these words, not that it matters since gonna search through with a while loop after COUNTDOWN_END

orient_only = 0
enc_only = 0
orient_enc = 0

for row in exp_df.itertuples(): #sub_df.itertuples():   
    sub = row.subject; session = row.session; exp = row.experiment
    mont = int(row.montage); loc = int(row.localization)
    reader = CMLReadDFRow(row)
    evs = reader.load('task_events')

    if (len(evs[evs.type=='ORIENT'])>0) & (len(evs[evs.type=='ENCODING_START'])>0):
        orient_enc+=1
    elif len(evs[evs.type=='ORIENT'])>0:
        orient_only+=1
    elif len(evs[evs.type=='ENCODING_START'])>0:
        enc_only+=1
            
orient_enc
orient_only
enc_only

In [7]:
## NOTE: ended up having to do this for each list individually since it's not a consistent offset across lists 
## (e.g. sometimes COUNTDOWN_END code shows up after ORIENT_START). See SWRanalysis from 2021-09-07.

# testing if loop works

after_countdown = []
check_sp = []

for row in exp_df.itertuples(): #sub_df.itertuples():   
    try:
        sub = row.subject; session = row.session; exp = row.experiment
        mont = int(row.montage); loc = int(row.localization)
        reader = CMLReadDFRow(row)
        evs = reader.load('task_events')
        
        evs_enc = evs[evs.list > -1] # remove practice lists **            
        evs_enc = evs_enc[evs_enc.eegoffset > -1] # any trial with no eeg gets removed by cmlreaders so it's not in ripple_array 
        
        got_words = 0
        next_type = 1
        while got_words == 0:
            # if you find a WORD, align all the events to this index to get the encoding_starts
            if np.unique(evs_enc.iloc[findInd(evs_enc.type=='COUNTDOWN_END')+next_type].type) == 'WORD':
                evs_enc = evs_enc.iloc[findInd(evs_enc.type=='COUNTDOWN_END')+next_type]
                # reset values
                got_words = 1
                next_type = 1
            else:
                next_type+=1

        after_countdown.append(np.unique(evs_enc.type))
        check_sp.append(np.unique(evs_enc.serialpos))
            
    except:
        a=1

In [8]:
#nailed it
after_countdown
check_sp

[array(['WORD'], dtype=object),
 array(['WORD'], dtype=object),
 array(['WORD'], dtype=object),
 array(['WORD'], dtype=object),
 array(['WORD'], dtype=object),
 array(['WORD'], dtype=object),
 array(['WORD'], dtype=object),
 array(['WORD'], dtype=object),
 array(['WORD'], dtype=object),
 array(['WORD'], dtype=object),
 array(['WORD'], dtype=object),
 array(['WORD'], dtype=object),
 array(['WORD'], dtype=object),
 array(['WORD'], dtype=object),
 array(['WORD'], dtype=object),
 array(['WORD'], dtype=object),
 array(['WORD'], dtype=object),
 array(['WORD'], dtype=object),
 array(['WORD'], dtype=object),
 array(['WORD'], dtype=object),
 array(['WORD'], dtype=object),
 array(['WORD'], dtype=object),
 array(['WORD'], dtype=object),
 array(['WORD'], dtype=object),
 array(['WORD'], dtype=object),
 array(['WORD'], dtype=object),
 array(['WORD'], dtype=object),
 array(['WORD'], dtype=object),
 array(['WORD'], dtype=object),
 array(['WORD'], dtype=object),
 array(['WORD'], dtype=object),
 array([

[array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),